# SQuaSH InfluxDB data model

In [DM-16775](https://github.com/lsst-sqre/squash-restful-api/tree/master/examples/influxdb_data_model.ipynb) we revisit the SQuaSH InfluxDB data model. You can use this notebook to try different strategies mapping SQuaSH data to InfluxDB. You can also use it to "manually" synchronize the SQuaSH production database with an InfluxDB instance. For a quick introduction on InfluxDB concepts see [this notebook](https://github.com/lsst-sqre/influx-demo).

In [ ]:
SQUASH_API_URL = "https://squash-restful-api-demo.lsst.codes/"
INFLUXDB_API_URL = "https://influxdb-demo.lsst.codes"

The following cells will grab SQuaSH data and write it in the format used by InfluxDB, the so called [line protocol](https://docs.influxdata.com/influxdb/v1.6/write_protocols/line_protocol_tutorial/):


```#<measurement>[,<tag_key>=<tag_value>[,<tag_key>=<tag_value>]] <field_key>=<field_value>[,<field_key>=<field_value>] [<timestamp>]```

Important InfluxDB concepts: an InfluxDB measurement is equivalent to an SQL table, tags are annotations that are used to query the data, and thus are indexed in InfluxDB. Fields correspond to the actual values (metric values, ci_id, run_id, etc.) and are not indexed. InfluxDB is optimized for time-series data which are indexed and sharded by the timestamp.

## Mapping lsst.verify concepts to InfluxDB

`lsst.verify` concepts are mapped as follow:

* lsst.verify package -> InfluxDB measurement
* lsst.verify metadata -> InfluxDB tag
* lsst.verify metric value - > InfluxDB field
* CI or LDF pipeline runtime -> InfluxDB timestamp

## Mapping SQuaSH metadata to InfluxDB
`lsst.verify` adds metadata to the verification jobs uploaded to SQuaSH. The mapping of these metadata to either InfluxDB tags or fields is defined here:

1. By default, if the SQuaSH metadata key is not found in the mapping, the original key will be preserved and it will be written to InfluxDB as a tag.
2. You can use the mapping to rename the metadata key when appropriate. 
3. Finally, if `schema` is set to `None`, then the corresponding metadata won't be written to InfluxDB.

In [176]:
MAPPING = [{'squash': 'ci_id',  # The ID of the execution environment is mapped to run_id
            'influxdb': 'run_id', 
            'schema': 'field'
           },
           {'squash': 'run_id',
            'influxdb': 'run_id',
            'schema': 'field'
           },
            {'squash': 'id',  
            'influxdb': 'squash_id',
            'schema': 'field'
           },
            {'squash': 'url',
             'influxdb': 'squash_url',
             'schema': 'field',
             'transformation': "format_url(data['id'], value)"
            },
           {'squash': 'run_id_url', # The URL of the execution environment is mapped to run_url
            'influxdb': 'run_url',
            'schema': 'field',
            'transformation': "format_url(data['run_id'], value)"
           },
           {'squash': 'ci_url',
            'influxdb': 'run_url',
            'schema': 'field',
            'transformation': "format_url(data['ci_id'], value)"
           },
           {'squash': 'ci_dataset', # The processed dataset is mapped to dataset
            'influxdb': 'dataset',
            'schema': 'tag'
           },
           {'squash': 'version_tag', 
            'influxdb': 'version_tag',
            'schema': 'field'
           },
           {'squash': 'filter_name',
            'influxdb': 'filter',
            'schema': 'tag'
           },
           {'squash': 'date_created',
            'influxdb': 'timestamp',
            'schema': 'field',
            'transformation': 'format_timestamp(value)'
           },
           {'squash': 'date', 
            'influxdb': None,
            'schema': None
           },
           {'squash': 'ci_label',
            'influxdb': None,
            'schema': None
           },
           {'squash': 'ci_name',
            'influxdb': 'pipeline',
            'schema': 'tag'
           },
           {'squash': 'code_changes',
            'influxdb': 'code_changes',
            'schema': 'field',
            'transformation': "format_code_changes(data['ci_id'])"
           },
           
           {'squash': 'code_changes_counts',
            'influxdb': 'code_changes_counts',
            'schema': 'field',
            'transformation': "format_code_changes_counts(data['ci_id'])"
           },
           {'squash': 'packages',
            'influxdb': None,
            'schema': None}
          ]

The rationale for this mapping is the following:
1. `ci_id`, `run_id` and `squash_id` are sequential ids. `ci_url` and `run_id_url` are different for each run. The reason for adding them as InfluxDB fields is to [reduce InfluxDB series cardinality](https://docs.influxdata.com/influxdb/v1.7/concepts/schema_and_data_layout/#discouraged-schema-design) (DM-18342) and we also want to display these information in the dashboards.
2. We define a common datamodel mapping variables from different execution environments example `ci_id` and `run_id` to `run_id`, `ci_url` and `run_id_url` to `run_url`, `ci_dataset` and `dataset` to `dataset`. The corresponding values can be filtered by the `env_name` tag. 
3. It is not possible to do [math operations with InfluxDB timestamps](https://community.influxdata.com/t/math-operations-on-field-value-and-time/6323/4) so it is useful to add the `timestamp` explicitly as a field. There's also a `date` field which is added as environment metadata that we don't need in InfluxDB (DM-17049)
4. `lsst.verify` metadata uses `filter_name`. We decided to rename it since it is commonly called filter in the dataID used in processing with DM software.
5. `ci_label` does not seem important so we skip that.
6. `ci_name` is mapped to `pipeline` to identify which pipeline ran that. That's important because some pipeline can have several verication packages which will be mapped to different InfluxDB measurements so we might want to group them.
7. We add the code changes information for pipelines that run in the Jenkins environment.
8. We also skip `packages` metadata for now. 
9. Other metadata not listed in the mapping are automatically added as InfluxDB tags.


See also [InfluxDB schema design and data layout](https://docs.influxdata.com/influxdb/v1.7/concepts/schema_and_data_layout/#general-recommendations) recommendations.



We start by creating a new InfluxDB database. If the database already exists, an status code 200 (OK) is returned, and the existing data is preserved. If you want to overwrite an existing database you have to delete it first using the Chronograf admin interface.

In [177]:
import requests
import json

INFLUXDB_DATABASE = "squash-demo"

params={'q': 'CREATE DATABASE "{}"'.format(INFLUXDB_DATABASE)}
r = requests.post(url=INFLUXDB_API_URL + "/query", params=params)
r.status_code

200

In [178]:
from pytz import UTC
from datetime import datetime
from dateutil.parser import parse

def format_timestamp(date):
    """ Format a timestamp string to be used in the InfluxDB line protocol.

        Parameters
        ----------
        date: `<str>`
            Timestamp string, e.g. 2019-02-11T19:06:32Z

        Returns
        -------
        timestamp: `<int>`
            Timestamp in nanosecond-precision Unix time.
            See https://docs.influxdata.com/influxdb/v1.6/write_protocols/
    """

    epoch = UTC.localize(datetime.utcfromtimestamp(0))

    timestamp = int((parse(date) - epoch).total_seconds() * 1e9)

    return timestamp


In [179]:
def format_url(title, url):
    return "[{}]({})".format(title, url)

In [180]:
def format_code_changes(ci_id):

    data = requests.get(SQUASH_API_URL + "/code_changes?ci_id={}".format(ci_id)).json()
    
    code_changes = []
    for pkg in data['packages']:
        name = pkg[0]
        git_sha = pkg[1]
        git_url = pkg[2]
        git_commit_url = git_url.replace('.git','/commit/')+git_sha
        code_changes.append(format_url(name, git_commit_url))
        
    return ", ".join(code_changes)

In [181]:
def format_code_changes_counts(ci_id):
    data = requests.get(SQUASH_API_URL + "/code_changes?ci_id={}".format(ci_id)).json()
    
    code_changes_counts = 0
    if data['counts']:
        code_changes_counts = data['counts']
    
    return code_changes_counts
    
    
    

In [182]:
def mapping(key):
    """ Perform the mapping between SQuaSH metadata and InfluxDB 
        given a MAPPING.
    
        Parameters
        ---------- 
        key: `str`
            The key to look for in the MAPPING.
        
        Returns
        -------
        mapped_key: `str` or `None`       
            Returns the `mapped_key` if the key is found in the MAPPING or the 
            original key if the key does not match.
       
        schema: `str` or `None`
            The InfluxDB schema to write, or `None` if the key should not 
            be added to InfluxDB. 
        
        
    """ 
    schema = 'tag'
    mapped_key = key
    transformation = None
    
    for m in MAPPING:
        if m['squash'] == key:
            mapped_key = m['influxdb']
            schema = m['schema']
            if 'transformation' in m:
                transformation = m['transformation']
            break
                
    return schema, mapped_key, transformation

In [183]:
def sanitize(obj):
    """ Return a valid string representing a tag key, a tag value or a field key.
        
        See https://docs.influxdata.com/influxdb/v0.13/write_protocols/
        write_syntax/#escaping-characters
    
        Parameters
        ----------
        obj: `<obj>`
            An object for the tag key, tag value or field key.
        
        Returns
        -------
        string: `str`
            A valid string for the tag key, tag value or field key.
    """
    string = str(obj)
    string = string.replace(" ", "\ ")
    string = string.replace(",", "\," )
    string = string.replace("=", "\=")
            
    return string

In [184]:
def process_metadata(data):
    """ Process SQuaSH metadata using a pre-configured mapping to InfluxDB.
    
        Parameters
        ----------
        data: `dict`
            A dictionary with SQuaSH metadata.
       
        Return
        ------
        tags: `<list>` 
            List of tags to be written to InfluxDB.
        fields: `<list>`
            List of fields to be written to InfluxDB.
    """
    tags = []
    fields = []
    for key, value in data.items():
        # process nested dict
        if isinstance(value, dict):
            tmp_tags, tmp_fields = process_metadata(value)
            tags.extend(tmp_tags)
            fields.extend(tmp_fields)
        else:
            schema, mapped_key, transformation = mapping(key)
            if transformation:
                value = eval(transformation)
            if mapped_key and schema == 'tag':
                tags.append("{}={}".format(sanitize(mapped_key), sanitize(value)))
            elif mapped_key and schema == 'field':
                if isinstance(value, str):
                    fields.append("{}=\"{}\"".format(sanitize(mapped_key), value))
                else:
                    fields.append("{}={}".format(sanitize(mapped_key), value))
    
    return tags, fields

In [185]:
def format_influxdb_line(measurement, tags, fields, timestamp):
    """ Format a line following the InfluxDB line protocol.

        Parameters
        ----------
        measurement: `<str>`
            Name of the InfluxDB measurement
        tags: `<list>`
            A list of valid InfluxDB tags
        fields: `<list>`
            A list of valid InfluxDB fields
        timestamp: `int`
            A timestamp in nanosecond-precision Unix time.

        Returns
        -------
        influxdb_line: `<str>`
            An InfluxDB line as defined by the line protocol in
            https://docs.influxdata.com/influxdb/v1.6/write_protocols/
    """
    line = "{},{} {} {}".format(measurement, ",".join(tags), ",".join(fields),
                                timestamp)
    return line


In [186]:
def send_to_influxdb(influxdb_line):
    """ Send a line to an InfluxDB database. It assumes INFLUXDB_DATABASE already
        exists in InfluxDB.

        Parameters
        ----------
        influxdb_line: `<str>`
            An InfluxDB line as defined by the line protocol in
            https://docs.influxdata.com/influxdb/v1.6/write_protocols/

        Returns
        -------
        status_code: `<int>`
            Status code from the InfluxDB HTTP API.
        text: `<str>`
            Status message from the InfluxDB HTTP API.
    """
    params = {'db': INFLUXDB_DATABASE}
    r = requests.post(url=INFLUXDB_API_URL + "/write", params=params,
                      data=influxdb_line)

    return r.status_code, r.text

In [187]:
import math
import urllib.parse

def job_to_influxdb(data):
    """Unpack a SQuaSH job and send it to InfluxDB. 
    
        Parameters
        ----------
        data: `<dict>`
            A dictionary containing the job data
        
        Returns
        -------
        status_code: `<int>`
             204:
               The request was processed successfully
             400:
               Malformed syntax or bad query

        Note
        ----
        `lsst.verify` measurement and InfluxDB measurement mean different things. 
    """    
    # This still gets the timestamp of an individual `lsst.verify` job, we want 
    # the timestamp of the Jenkins job instead.
    # DM-18359 - SQuaSH API /jenkins/<ci_id> should return the jenkins timestamp 
    
    timestamp = format_timestamp(data['date_created'])
    
    # Add extra metadata
    
    data['meta']['id'] = data['id']
    
    data['meta']['url'] = urllib.parse.urljoin(SQUASH_API_URL, 
                                               "/job/{}".format(data['id']))

    data['meta']['date_created'] = data['date_created']
    data['meta']['env']['ci_dataset'] = data['ci_dataset']
    
    # Edge cases that we cannot handle in the mapping
    
    # Fix dataset_repo_url duplication
    if 'dataset_repo_url' in data['meta'].keys():
        del data['meta']['dataset_repo_url']
    
    # Fix use of ci_dataset key in environments other than jenkins
    if data['meta']['env']['env_name'] != 'jenkins':
        if 'ci_dataset' in data['meta']['env']:
            del data['meta']['env']['ci_dataset']
            
    # add code changes 
    if data['meta']['env']['env_name'] == 'jenkins':
        data['meta']['env']['code_changes'] = ''
        data['meta']['env']['code_changes_counts'] = ''
    
    tags, extra_fields = process_metadata(data['meta'])
    
    
    # `lsst.verify` package -> InfluxDB measurement
    # `lsst.verify` metric value -> InfluxDB field
    # Group InfluxDB fields by the corresponding InfluxDB measurement
    
    fields_by_measurement = {}
    for verify_measurement in data['measurements']:
        # DM-18360 - SQuaSH API /measurements should return the verification package 
        influxdb_measurement = verify_measurement['metric'].split('.')[0]

        if influxdb_measurement not in fields_by_measurement:
            fields_by_measurement[influxdb_measurement] = []
            
        # InfluxDB does not store NaNs
        # https://github.com/influxdata/influxdb/issues/4089
        if not math.isnan(verify_measurement['value']):
            fields_by_measurement[influxdb_measurement].append("{}={}".format(verify_measurement['metric'],
                                                                              verify_measurement['value']))
    
    # By grouping InfluxDB fields we can also send all fields that belong to a 
    # measurement at once.
    for influxdb_measurement in fields_by_measurement:
    
        fields = fields_by_measurement[influxdb_measurement] + extra_fields
        influxdb_line = format_influxdb_line(influxdb_measurement, tags, fields,
                                             timestamp)

        status_code, message = send_to_influxdb(influxdb_line)
        if status_code != 204:
            print(message)

    return 

Retrieve a list of verification jobs from SQuaSH and send them to InfluxDB. As you run this notebook you might follow the data being written to InfluxDB using the [Data Explorer tool](https://chronograf-demo.lsst.codes/) in Chronograf. 



In [188]:
jobs = requests.get(SQUASH_API_URL + "/jobs").json()

for job_id in jobs['ids']:
    
    data = requests.get(SQUASH_API_URL + "/job/{}".format(job_id)).json()
    
    # Skip deprecated datasets
    if data['ci_dataset'] == 'unknown' or data['ci_dataset'] == 'decam':
        continue

    print('Sending InfluxDB line for job {}.'.format(job_id))
    
    job_to_influxdb(data)
    

Sending InfluxDB line for job 1.
Sending InfluxDB line for job 3.
Sending InfluxDB line for job 5.
Sending InfluxDB line for job 7.
Sending InfluxDB line for job 9.
Sending InfluxDB line for job 11.
Sending InfluxDB line for job 13.
Sending InfluxDB line for job 14.
Sending InfluxDB line for job 15.
Sending InfluxDB line for job 16.
Sending InfluxDB line for job 17.
Sending InfluxDB line for job 18.
Sending InfluxDB line for job 19.
Sending InfluxDB line for job 20.
Sending InfluxDB line for job 21.
Sending InfluxDB line for job 22.
Sending InfluxDB line for job 23.
Sending InfluxDB line for job 24.
Sending InfluxDB line for job 25.
Sending InfluxDB line for job 26.
Sending InfluxDB line for job 27.
Sending InfluxDB line for job 28.
Sending InfluxDB line for job 29.
Sending InfluxDB line for job 30.
Sending InfluxDB line for job 31.
Sending InfluxDB line for job 32.
Sending InfluxDB line for job 33.
Sending InfluxDB line for job 34.
Sending InfluxDB line for job 35.
Sending InfluxDB li

Sending InfluxDB line for job 244.
Sending InfluxDB line for job 245.
Sending InfluxDB line for job 246.
Sending InfluxDB line for job 247.
Sending InfluxDB line for job 248.
Sending InfluxDB line for job 249.
Sending InfluxDB line for job 250.
Sending InfluxDB line for job 251.
Sending InfluxDB line for job 252.
Sending InfluxDB line for job 253.
Sending InfluxDB line for job 254.
Sending InfluxDB line for job 255.
Sending InfluxDB line for job 256.
Sending InfluxDB line for job 257.
Sending InfluxDB line for job 258.
Sending InfluxDB line for job 259.
Sending InfluxDB line for job 260.
Sending InfluxDB line for job 261.
Sending InfluxDB line for job 262.
Sending InfluxDB line for job 263.
Sending InfluxDB line for job 264.
Sending InfluxDB line for job 265.
Sending InfluxDB line for job 266.
Sending InfluxDB line for job 267.
Sending InfluxDB line for job 268.
Sending InfluxDB line for job 269.
Sending InfluxDB line for job 270.
Sending InfluxDB line for job 271.
Sending InfluxDB lin

Sending InfluxDB line for job 479.
Sending InfluxDB line for job 480.
Sending InfluxDB line for job 481.
Sending InfluxDB line for job 482.
Sending InfluxDB line for job 483.
Sending InfluxDB line for job 484.
Sending InfluxDB line for job 485.
Sending InfluxDB line for job 486.
Sending InfluxDB line for job 487.
Sending InfluxDB line for job 488.
Sending InfluxDB line for job 489.
Sending InfluxDB line for job 490.
Sending InfluxDB line for job 491.
Sending InfluxDB line for job 492.
Sending InfluxDB line for job 493.
Sending InfluxDB line for job 494.
Sending InfluxDB line for job 495.
Sending InfluxDB line for job 496.
Sending InfluxDB line for job 497.
Sending InfluxDB line for job 498.
Sending InfluxDB line for job 499.
Sending InfluxDB line for job 500.
Sending InfluxDB line for job 501.
Sending InfluxDB line for job 502.
Sending InfluxDB line for job 503.
Sending InfluxDB line for job 504.
Sending InfluxDB line for job 505.
Sending InfluxDB line for job 506.
Sending InfluxDB lin

Sending InfluxDB line for job 717.
Sending InfluxDB line for job 718.
Sending InfluxDB line for job 719.
Sending InfluxDB line for job 720.
Sending InfluxDB line for job 721.
Sending InfluxDB line for job 722.
Sending InfluxDB line for job 723.
Sending InfluxDB line for job 724.
Sending InfluxDB line for job 725.
Sending InfluxDB line for job 726.
Sending InfluxDB line for job 727.
Sending InfluxDB line for job 728.
Sending InfluxDB line for job 729.
Sending InfluxDB line for job 730.
Sending InfluxDB line for job 731.
Sending InfluxDB line for job 732.
Sending InfluxDB line for job 733.
Sending InfluxDB line for job 734.
Sending InfluxDB line for job 735.
Sending InfluxDB line for job 736.
Sending InfluxDB line for job 737.
Sending InfluxDB line for job 738.
Sending InfluxDB line for job 739.
Sending InfluxDB line for job 740.
Sending InfluxDB line for job 741.
Sending InfluxDB line for job 742.
Sending InfluxDB line for job 743.
Sending InfluxDB line for job 744.
Sending InfluxDB lin

Sending InfluxDB line for job 1042.
Sending InfluxDB line for job 1043.
Sending InfluxDB line for job 1044.
Sending InfluxDB line for job 1045.
Sending InfluxDB line for job 1046.
Sending InfluxDB line for job 1047.
Sending InfluxDB line for job 1048.
Sending InfluxDB line for job 1049.
Sending InfluxDB line for job 1050.
Sending InfluxDB line for job 1051.
Sending InfluxDB line for job 1052.
Sending InfluxDB line for job 1053.
Sending InfluxDB line for job 1054.
Sending InfluxDB line for job 1055.
Sending InfluxDB line for job 1056.
Sending InfluxDB line for job 1057.
Sending InfluxDB line for job 1058.
Sending InfluxDB line for job 1059.
Sending InfluxDB line for job 1060.
Sending InfluxDB line for job 1061.
Sending InfluxDB line for job 1062.
Sending InfluxDB line for job 1063.
Sending InfluxDB line for job 1064.
Sending InfluxDB line for job 1065.
Sending InfluxDB line for job 1066.
Sending InfluxDB line for job 1067.
Sending InfluxDB line for job 1068.
Sending InfluxDB line for jo

...

Sending InfluxDB line for job 2739.
Sending InfluxDB line for job 2740.
Sending InfluxDB line for job 2741.
Sending InfluxDB line for job 2742.
Sending InfluxDB line for job 2743.
Sending InfluxDB line for job 2744.
Sending InfluxDB line for job 2745.
Sending InfluxDB line for job 2746.
Sending InfluxDB line for job 2747.
Sending InfluxDB line for job 2748.
Sending InfluxDB line for job 2749.
Sending InfluxDB line for job 2750.
Sending InfluxDB line for job 2751.
Sending InfluxDB line for job 2752.
Sending InfluxDB line for job 2753.
Sending InfluxDB line for job 2754.
Sending InfluxDB line for job 2755.
Sending InfluxDB line for job 2756.
Sending InfluxDB line for job 2757.
Sending InfluxDB line for job 2758.
Sending InfluxDB line for job 2759.
Sending InfluxDB line for job 2760.
Sending InfluxDB line for job 2761.
Sending InfluxDB line for job 2762.
Sending InfluxDB line for job 2763.
Sending InfluxDB line for job 2764.
Sending InfluxDB line for job 2765.
Sending InfluxDB line for jo

Sending InfluxDB line for job 2967.
Sending InfluxDB line for job 2968.
Sending InfluxDB line for job 2969.
Sending InfluxDB line for job 2970.
Sending InfluxDB line for job 2971.
Sending InfluxDB line for job 2972.
Sending InfluxDB line for job 2973.
Sending InfluxDB line for job 2974.
Sending InfluxDB line for job 2975.
Sending InfluxDB line for job 2976.
Sending InfluxDB line for job 2977.
Sending InfluxDB line for job 2978.
Sending InfluxDB line for job 2979.
Sending InfluxDB line for job 2980.
Sending InfluxDB line for job 2981.
Sending InfluxDB line for job 2982.
Sending InfluxDB line for job 2983.
Sending InfluxDB line for job 2984.
Sending InfluxDB line for job 2985.
Sending InfluxDB line for job 2986.
Sending InfluxDB line for job 2987.
Sending InfluxDB line for job 2988.
Sending InfluxDB line for job 2989.
Sending InfluxDB line for job 2990.
Sending InfluxDB line for job 2991.
Sending InfluxDB line for job 2992.
Sending InfluxDB line for job 2993.
Sending InfluxDB line for jo

Sending InfluxDB line for job 3201.
Sending InfluxDB line for job 3202.
Sending InfluxDB line for job 3203.
Sending InfluxDB line for job 3204.
Sending InfluxDB line for job 3205.
Sending InfluxDB line for job 3206.
Sending InfluxDB line for job 3207.
Sending InfluxDB line for job 3208.
Sending InfluxDB line for job 3209.
Sending InfluxDB line for job 3210.
Sending InfluxDB line for job 3211.
Sending InfluxDB line for job 3212.
Sending InfluxDB line for job 3213.
Sending InfluxDB line for job 3214.
Sending InfluxDB line for job 3215.
Sending InfluxDB line for job 3216.
Sending InfluxDB line for job 3217.
Sending InfluxDB line for job 3218.
Sending InfluxDB line for job 3219.
Sending InfluxDB line for job 3220.
Sending InfluxDB line for job 3221.
Sending InfluxDB line for job 3222.
Sending InfluxDB line for job 3223.
Sending InfluxDB line for job 3224.
Sending InfluxDB line for job 3225.
Sending InfluxDB line for job 3226.
Sending InfluxDB line for job 3227.
Sending InfluxDB line for jo

Sending InfluxDB line for job 3429.
Sending InfluxDB line for job 3430.
Sending InfluxDB line for job 3431.
Sending InfluxDB line for job 3432.
Sending InfluxDB line for job 3433.
Sending InfluxDB line for job 3434.
Sending InfluxDB line for job 3435.
Sending InfluxDB line for job 3436.
Sending InfluxDB line for job 3437.
Sending InfluxDB line for job 3438.
Sending InfluxDB line for job 3439.
Sending InfluxDB line for job 3440.
Sending InfluxDB line for job 3441.
Sending InfluxDB line for job 3442.
Sending InfluxDB line for job 3443.
Sending InfluxDB line for job 3444.
Sending InfluxDB line for job 3445.
Sending InfluxDB line for job 3446.
Sending InfluxDB line for job 3447.
Sending InfluxDB line for job 3448.
Sending InfluxDB line for job 3449.
Sending InfluxDB line for job 3450.
Sending InfluxDB line for job 3451.
Sending InfluxDB line for job 3452.
Sending InfluxDB line for job 3453.
Sending InfluxDB line for job 3454.
Sending InfluxDB line for job 3455.
Sending InfluxDB line for jo

Sending InfluxDB line for job 3663.
Sending InfluxDB line for job 3664.
Sending InfluxDB line for job 3665.
Sending InfluxDB line for job 3666.
Sending InfluxDB line for job 3667.
Sending InfluxDB line for job 3668.
Sending InfluxDB line for job 3669.
Sending InfluxDB line for job 3670.
Sending InfluxDB line for job 3671.
Sending InfluxDB line for job 3672.
Sending InfluxDB line for job 3673.
Sending InfluxDB line for job 3674.
Sending InfluxDB line for job 3675.
Sending InfluxDB line for job 3676.
Sending InfluxDB line for job 3677.
Sending InfluxDB line for job 3678.
Sending InfluxDB line for job 3679.
Sending InfluxDB line for job 3680.
Sending InfluxDB line for job 3681.
Sending InfluxDB line for job 3682.
Sending InfluxDB line for job 3683.
Sending InfluxDB line for job 3684.
Sending InfluxDB line for job 3685.
Sending InfluxDB line for job 3686.
Sending InfluxDB line for job 3687.
Sending InfluxDB line for job 3688.
Sending InfluxDB line for job 3689.
Sending InfluxDB line for jo